# Functional Interface

pytask offers a functional interface to users who want more flexibility than is given by a command line interface. It even allows you to run pytask from a Python interpreter or a Jupyter notebook like this article here.


Let's see how it works!

In [1]:
from pathlib import Path
from typing import Annotated

import pytask
from pytask import task

Here is a small workflow where two tasks create two text files and the third task merges both of them into one file.

One important bit to note here is that the second task is created from a lambda function. So, you can use dynamically defined functions to create tasks.

It also shows how easy it is to wrap any third-party function where you have no control over the signature, but you can still easily wrap them with pytask.

In [2]:
def task_create_first_file() -> Annotated[str, Path("first.txt")]:
    return "Hello, "


task_create_second_file = task(
    name="task_create_second_file", produces=Path("second.txt")
)(lambda *x: "World!")


def task_merge_files(
    first: Path = Path("first.txt"), second: Path = Path("second.txt")
) -> Annotated[str, Path("hello_world.txt")]:
    return first.read_text() + second.read_text()

Now, let us execute this little workflow.

In [3]:
session = pytask.build(
    tasks=[task_create_first_file, task_merge_files, task_create_second_file]
)

────────────────────────────────────────────── Start pytask session ───────────────────────────────────────────────

Platform: darwin -- Python 3.12.12, pytask 0.5.6, pluggy 1.6.0

Root: /Users/tobiasr/git/pytask

Collected 3 tasks.

Output()

╭─────────────────────────┬─────────╮
│ Task                    │ Outcome │
├─────────────────────────┼─────────┤
│ ]8;id=73604;file://None\task_create_first_file ]8;;\ │ .       │
│ ]8;id=609946;file://None\task_create_second_file]8;;\ │ .       │
│ ]8;id=378329;file://None\task_merge_files       ]8;;\ │ .       │
╰─────────────────────────┴─────────╯

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────── Summary ────────────╮
│  3  Collected tasks            │
│  3  Succeeded        (100.0%)  │
╰────────────────────────────────╯

──────────────────────────────────────────── Succeeded in 0.03 seconds ────────────────────────────────────────────

The information on the executed workflow can be found in the `session`.

In [4]:
session

Session(config={'pm': <pluggy._manager.PluginManager object at 0x1116ec6e0>, 'markers': {'filterwarnings': 'Add a filter for a warning to a task.', 'persist': 'Prevent execution of a task if all products exist and even if something has changed (dependencies, source file, products). This decorator might be useful for expensive tasks where only the formatting of the file has changed. The state of the files which have changed will also be remembered and another run will skip the task with success.', 'skip': 'Skip a task and all its dependent tasks.', 'skip_ancestor_failed': 'Internal decorator applied to tasks if any of its preceding tasks failed.', 'skip_unchanged': 'Internal decorator applied to tasks which have already been executed and have not been changed.', 'skipif': 'Skip a task and all its dependent tasks if a condition is met.', 'task': 'Mark a function as a task regardless of its name. Or mark tasks which are repeated in a loop. See this tutorial for more information: [link htt

## Repeated Tasks

You can also create multiple tasks with the same function by repeating the task in a loop. (Because we are collecting the tasks ourselves in a list, we don't necessarily need the `@task` decorator, but you can still use it.) This is useful when you want to run the same operation with different parameters.

In [5]:
from pytask import Product

tasks = []
for i in range(3):

    def create_file(
        value: int = i * 100, path: Annotated[Path, Product] = Path(f"output_{i}.txt")
    ) -> None:
        path.write_text(f"Result: {value}")

    tasks.append(create_file)

In [6]:
session = pytask.build(tasks=tasks)

────────────────────────────────────────────── Start pytask session ───────────────────────────────────────────────

Platform: darwin -- Python 3.12.12, pytask 0.5.6, pluggy 1.6.0

Root: /Users/tobiasr/git/pytask

Collected 3 tasks.

Output()

╭────────────────────────┬─────────╮
│ Task                   │ Outcome │
├────────────────────────┼─────────┤
│ ]8;id=133063;file://None\create_file[0-path0]  ]8;;\ │ .       │
│ ]8;id=167889;file://None\create_file[100-path1]]8;;\ │ .       │
│ ]8;id=365664;file://None\create_file[200-path2]]8;;\ │ .       │
╰────────────────────────┴─────────╯

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────── Summary ────────────╮
│  3  Collected tasks            │
│  3  Succeeded        (100.0%)  │
╰────────────────────────────────╯

──────────────────────────────────────────── Succeeded in 0.03 seconds ────────────────────────────────────────────

In [7]:
# Cleanup
for i in range(3):
    Path(f"output_{i}.txt").unlink()

## Configuring the build

To configure the build, {func}`pytask.build` has many more options that are the same that you find on the commandline.

In [8]:
pytask.build?

Signature:
pytask.build(
    *,
    capture: "Literal['fd', 'no', 'sys', 'tee-sys'] | CaptureMethod" = <CaptureMethod.FD: 'fd'>,
    check_casing_of_paths: 'bool' = True,
    config: 'Path | None' = None,
    database_url: 'str' = '',
    debug_pytask: 'bool' = False,
    disable_warnings: 'bool' = False,
    dry_run: 'bool' = False,
    editor_url_scheme: "Literal['no_link', 'file', 'vscode', 'pycharm'] | str" = 'file',
    explain: 'bool' = False,
    expression: 'str' = '',
    force: 'bool' = False,
    ignore: 'Iterable[str]' = (),
    marker_expression: 'str' = '',
    max_failures: 'float' = inf,
    n_entries_in_table: 'int' = 15,
    paths: 'Path | Iterable[Path]' = (),
    pdb: 'bool' = False,
    pdb_cls: 'str' = '',
    s: 'bool' = False,
    show_capture: "Literal['no', 'stdout', 'stderr', 'all'] | ShowCapture" = <ShowCapture.ALL: 'all'>,
    show_errors_immediately: 'bool' = False,
    show_locals: 'bool' = False,
    show_traceback: 'bool' = True,
    sort_table: 'bool' 

In [9]:
# Cleanup
for name in ("first.txt", "second.txt", "hello_world.txt"):
    Path(name).unlink()